In [1]:
!pip install dgl
from os import listdir
import dgl
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

     |████████████████████████████████| 3.0MB 3.5MB/s 


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
epochs = int(input("Enter number of epochs: "))
learning_rate = float(input("Enter learning rate: "))

Enter number of epochs: 250
Enter learning rate: 0.01


In [4]:
filename = "/content/drive/My Drive/adaptec1.nets"

edge_file = "/content/add_edges.py"
edge_file_handle = open(edge_file, "w+")

node_count = 0

with open(filename,encoding="utf8") as f:
    netSection = False
    edges = 0
    lineCount = 0
    src=[]
    dest=[]
    for line in f:
        lineCount += 1
        #Newline
        if len(line) < 2:
            continue
            
        # Comments
        if line[0] == '#':
            continue
        #print(line)
        splitLine = line.split()
        #print("Words = ", len(splitLine))
        if(splitLine[0] == 'NetDegree'):
            
            if edges > 0: #Previous NetDegree not complete!
                print("Missed an edge at line: ", lineCount-1)
                
            edges = int(splitLine[2])
            netSection = True
            src=[]
            dest=[]
            #G.add_nodes(num)
            
        elif netSection == True:
          node_name = splitLine[0][1:]
          node_val  = int(node_name)
          if node_count < node_val+1:
              node_count = node_val+1
          
          if(splitLine[1] == 'O'):
              src.append(node_name)
              edges -= 1
          elif(splitLine[1] == 'I'):
              dest.append(node_name)
              edges -= 1
      
        if netSection == True:
            if edges == 0:
                if (len(src)<1):
                    print("No source edge, skipping")
                    continue
                if (len(dest)<1):
                    print("No dest edge, skipping")
                    continue
                    
                netSection = False
                for des_idx in range(len(dest)):
                    cmd = "G.add_edges("+src[0]+","+dest[des_idx]+")"+"\n"
                    edge_file_handle.write(cmd)
                    
        #if lineCount > 100:
        #    break


edge_file_handle.close()


No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584552
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584660
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584723
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584753
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584781
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584978
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585078
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585138
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585199
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585306
No dest edge, skipping
No dest edge, skipping
Missed an edge at line: 

In [5]:
G = dgl.DGLGraph()
G.add_nodes(node_count)
exec(open(edge_file).read())
print("Nodes = ", G.number_of_nodes())
print("Edges = ", G.number_of_edges())

Nodes =  211447
Edges =  720134


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
embed = nn.Embedding(G.number_of_nodes(),6)

In [0]:
G.ndata['feat'] = embed.weight

In [0]:
from dgl.nn.pytorch import GraphConv

In [0]:
class GCN(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(GCN,self).__init__()
        self.conv1 = GraphConv(input_size,hidden_size)
        self.conv2 = GraphConv(hidden_size,num_classes)
    
    def forward(self,G,inputs):
        h = self.conv1(G,inputs)
        h = torch.relu(h) 
        h = self.conv2(G,h)
        return h

In [0]:
net = GCN(6,5,20)

In [0]:
inputs = embed.weight
a = np.linspace(0,G.number_of_nodes()-1,20,dtype=int)
labelled_nodes = torch.tensor(a)
labels = torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

# Training Model

In [13]:
import itertools
import time

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), embed.parameters()), lr=learning_rate)
all_logits = []

start_time = time.time()

for epoch in range(epochs):
    logits = net(G, inputs)
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[labelled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Epoch {} | Loss: {}'.format(epoch + 1, loss.item()))

end_time = time.time()
print("Training Time: {}".format(end_time - start_time))

Epoch 1 | Loss: 3.1068472862243652
Epoch 2 | Loss: 3.0820565223693848
Epoch 3 | Loss: 3.059337615966797
Epoch 4 | Loss: 3.0379719734191895
Epoch 5 | Loss: 3.017899513244629
Epoch 6 | Loss: 2.9990484714508057
Epoch 7 | Loss: 2.9810805320739746
Epoch 8 | Loss: 2.963773012161255
Epoch 9 | Loss: 2.9479899406433105
Epoch 10 | Loss: 2.9323067665100098
Epoch 11 | Loss: 2.916588068008423
Epoch 12 | Loss: 2.900881767272949
Epoch 13 | Loss: 2.885002851486206
Epoch 14 | Loss: 2.868860960006714
Epoch 15 | Loss: 2.8523755073547363
Epoch 16 | Loss: 2.83532977104187
Epoch 17 | Loss: 2.8175220489501953
Epoch 18 | Loss: 2.7989351749420166
Epoch 19 | Loss: 2.779670238494873
Epoch 20 | Loss: 2.7596089839935303
Epoch 21 | Loss: 2.7387399673461914
Epoch 22 | Loss: 2.7169833183288574
Epoch 23 | Loss: 2.694404125213623
Epoch 24 | Loss: 2.670832633972168
Epoch 25 | Loss: 2.646305561065674
Epoch 26 | Loss: 2.621131181716919
Epoch 27 | Loss: 2.595022678375244
Epoch 28 | Loss: 2.568037509918213
Epoch 29 | Loss: 

In [0]:
pos = {}
cluster_nodes = {} # Every value in key->value pair includes the cluster centre node as well.
for v in range(G.number_of_nodes()):
  pos[v] = all_logits[99][v].numpy()
  cls = pos[v].argmax()
  if cluster_nodes.get(cls) :
    cluster_nodes[cls].append(v)
  else:
    cluster_nodes[cls] = [] 
    cluster_nodes[cls].append(v)

In [15]:
print(cluster_nodes)

{0: [0, 119, 144, 674, 1249, 1579, 2290, 4737, 4739, 5019, 5322, 5475, 5620, 6479, 6860, 7331, 7397, 7458, 9369, 10584, 11435, 11444, 11471, 12293, 12296, 12300, 12875, 12878, 12892, 14224, 15202, 15584, 16366, 16500, 16813, 17064, 18145, 18154, 18169, 18669, 19122, 19160, 20223, 20235, 20692, 20731, 20779, 21698, 21740, 22934, 23422, 23892, 24402, 24894, 25313, 25493, 28297, 28320, 28389, 28472, 28960, 29300, 29445, 30053, 30948, 31479, 31630, 31814, 32568, 32577, 32858, 33317, 33611, 34573, 34980, 35032, 35072, 35115, 35133, 35526, 38332, 38657, 38715, 38717, 38719, 38736, 38745, 39628, 39735, 39749, 39849, 40388, 40448, 40835, 41027, 41142, 41683, 41716, 42506, 43523, 43788, 44344, 44411, 45585, 47155, 47167, 47178, 47217, 47218, 47222, 47393, 47409, 47436, 47599, 48741, 48765, 48819, 48941, 48970, 49073, 49555, 49707, 50032, 50119, 50565, 50625, 51312, 52933, 52991, 53027, 53060, 53081, 53576, 53706, 54007, 54205, 54619, 54862, 54891, 54898, 54903, 54907, 55224, 57608, 58494, 58504

# Cluster Quality Metric #2 : Number of Nodes per Cluster Distribution

In [0]:
def number_of_nodes_per_cluster():
  number_dict = {} # Dictionary mapping the node label to the number of nodes in a particular cluster
  for key,value in cluster_nodes.items():
    number_dict[key] = len(value)

  number_dict = sorted(number_dict.items()) # Sorting dictionary by keys in increasing order

  for ix in number_dict:
    print("Number of Nodes in Cluster {} = {}\n".format(ix[0],ix[1]))

  return number_dict

In [17]:
number_dict = number_of_nodes_per_cluster()

Number of Nodes in Cluster 0 = 507

Number of Nodes in Cluster 1 = 4508

Number of Nodes in Cluster 2 = 2109

Number of Nodes in Cluster 3 = 21089

Number of Nodes in Cluster 4 = 2517

Number of Nodes in Cluster 5 = 16926

Number of Nodes in Cluster 6 = 891

Number of Nodes in Cluster 7 = 3992

Number of Nodes in Cluster 8 = 32043

Number of Nodes in Cluster 9 = 2293

Number of Nodes in Cluster 10 = 1712

Number of Nodes in Cluster 11 = 9605

Number of Nodes in Cluster 12 = 19973

Number of Nodes in Cluster 13 = 5588

Number of Nodes in Cluster 14 = 9701

Number of Nodes in Cluster 15 = 31999

Number of Nodes in Cluster 16 = 3492

Number of Nodes in Cluster 17 = 34601

Number of Nodes in Cluster 18 = 3542

Number of Nodes in Cluster 19 = 4359



# HashMap (Dictionary) for easy visualization of networkx graph
- Dictionary to map node numbers in a particular cluster to a standard count procedure of 0 to (n-1) 
- n = number of nodes in a particular cluster
- Cluster label is taken as input from user

In [0]:
num = int(input("Enter cluster number: "))

Enter cluster number: 10


In [0]:
standard_nodes_dict = {}

count = 0

for node in cluster_nodes[num]:
  standard_nodes_dict[node] = count
  count += 1

In [0]:
print(standard_nodes_dict.items())

dict_items([(44, 0), (45, 1), (48, 2), (49, 3), (88, 4), (89, 5), (90, 6), (137, 7), (140, 8), (192, 9), (193, 10), (194, 11), (200, 12), (201, 13), (202, 14), (232, 15), (233, 16), (234, 17), (256, 18), (257, 19), (258, 20), (263, 21), (280, 22), (281, 23), (282, 24), (284, 25), (285, 26), (286, 27), (296, 28), (297, 29), (298, 30), (303, 31), (345, 32), (346, 33), (347, 34), (357, 35), (358, 36), (359, 37), (387, 38), (388, 39), (389, 40), (396, 41), (397, 42), (398, 43), (443, 44), (459, 45), (462, 46), (503, 47), (504, 48), (505, 49), (539, 50), (540, 51), (541, 52), (557, 53), (558, 54), (559, 55), (563, 56), (564, 57), (565, 58), (623, 59), (629, 60), (638, 61), (682, 62), (683, 63), (704, 64), (713, 65), (725, 66), (739, 67), (802, 68), (805, 69), (806, 70), (849, 71), (931, 72), (933, 73), (1060, 74), (1061, 75), (1068, 76), (1069, 77), (1113, 78), (1132, 79), (1135, 80), (1152, 81), (1156, 82), (1236, 83), (1242, 84), (1295, 85), (1296, 86), (1301, 87), (1302, 88), (1321, 89),

# Clustering Visualisation: Cluster Quality Metric # 3
- Visualisation for any generic cluster label taken as input from user

In [0]:
# dict_keys = list(standard_nodes_dict.keys())

# input_edges = G.in_edges(dict_keys, form='uv')

# ax = input_edges[0].numpy()

# ax = [i for i in ax if i in standard_nodes_dict.keys()]

# print(ax)

In [0]:
src = []
dst = []

In [0]:
def build_graph():

  dict_keys = list(standard_nodes_dict.keys())

  input_edges = G.in_edges(dict_keys, form='uv')

  src = input_edges[0].numpy()
  dst = input_edges[1].numpy()

  m = src.shape[0]
  src1 = []
  dst1 = []

  for ix in range(m):
    if src[ix] not in standard_nodes_dict.keys():
      continue
    else:
      src1.append(src[ix])
      dst1.append(dst[ix])

  src1 = np.array(src1)
  dst1 = np.array(dst1)

  j = 0
  for num in src1:
    num = standard_nodes_dict.get(num)
    src1[j] = num
    j += 1

  i = 0
  for num1 in dst1:
      num1 = standard_nodes_dict[num1]
      dst1[i] = num1
      i += 1

  u = np.concatenate([src1,dst1])
  v = np.concatenate([dst1,src1])

  return dgl.DGLGraph((u,v))

In [0]:
G1 = build_graph()

In [0]:
print("Nodes = {}".format(G1.number_of_nodes()))
print("Edges = {}".format(G1.number_of_edges()))

Nodes = 14718
Edges = 14738


In [0]:
nx_G1 = G1.to_networkx().to_undirected()

In [0]:
# nx.write_edgelist(nx_G1,"/content/cluster14.csv")

In [0]:
# pos1 = nx.kamada_kawai_layout(nx_G1) # Session crashes on running this cell

In [0]:
# plt.style.use("seaborn")

In [0]:
# nx.draw(nx_G1,pos1,with_labels=True,node_color=[[0.7,0.7,0.7]])
# plt.show()

# Cluster Quality Metric # 1: Total number of edges between nodes which belong to different clusters divided by total edges represented as percentage (%)

In [18]:
m = 10 # Total number of clusters

edges_between_clusters = {}

for i in range(m):
  print("Calculating for cluster {}\n".format(i))
  nodes_i = cluster_nodes[i]

  for j in range(i+1,m):
    nodes_j = np.array(cluster_nodes[j])

    ij_input_edges = G.in_edges(nodes_i,form='uv')
    edges_src = np.array(ij_input_edges[0])
    
    count = 0

    for val1 in nodes_j:
      for val2 in edges_src:
        if val1 == val2:
          count += 1

    # for x in range(np.array(nodes_i).shape[0]):
    #   if edges_src[x] == nodes_j[x]:
    #     count += 1

    edges_between_clusters[(i,j)] = count

Calculating for cluster 0

Calculating for cluster 1

Calculating for cluster 2

Calculating for cluster 3

Calculating for cluster 4

Calculating for cluster 5

Calculating for cluster 6

Calculating for cluster 7

Calculating for cluster 8

Calculating for cluster 9



In [19]:
edges_between_clusters

{(0, 1): 26,
 (0, 2): 11,
 (0, 3): 80,
 (0, 4): 28,
 (0, 5): 64,
 (0, 6): 7,
 (0, 7): 17,
 (0, 8): 90,
 (0, 9): 14,
 (1, 2): 108,
 (1, 3): 1100,
 (1, 4): 137,
 (1, 5): 851,
 (1, 6): 51,
 (1, 7): 167,
 (1, 8): 1462,
 (1, 9): 117,
 (2, 3): 505,
 (2, 4): 83,
 (2, 5): 373,
 (2, 6): 23,
 (2, 7): 71,
 (2, 8): 606,
 (2, 9): 75,
 (3, 4): 564,
 (3, 5): 4439,
 (3, 6): 327,
 (3, 7): 1072,
 (3, 8): 9938,
 (3, 9): 465,
 (4, 5): 419,
 (4, 6): 18,
 (4, 7): 90,
 (4, 8): 567,
 (4, 9): 87,
 (5, 6): 289,
 (5, 7): 941,
 (5, 8): 9799,
 (5, 9): 401,
 (6, 7): 15,
 (6, 8): 130,
 (6, 9): 11,
 (7, 8): 1874,
 (7, 9): 94,
 (8, 9): 509}

In [0]:
percentage_dict = {}

for key,value in edges_between_clusters.items():
  value = (value/G.number_of_edges())*100
  value = round(value,4)
  percentage_dict[key] = value

In [21]:
percentage_dict

{(0, 1): 0.0036,
 (0, 2): 0.0015,
 (0, 3): 0.0111,
 (0, 4): 0.0039,
 (0, 5): 0.0089,
 (0, 6): 0.001,
 (0, 7): 0.0024,
 (0, 8): 0.0125,
 (0, 9): 0.0019,
 (1, 2): 0.015,
 (1, 3): 0.1527,
 (1, 4): 0.019,
 (1, 5): 0.1182,
 (1, 6): 0.0071,
 (1, 7): 0.0232,
 (1, 8): 0.203,
 (1, 9): 0.0162,
 (2, 3): 0.0701,
 (2, 4): 0.0115,
 (2, 5): 0.0518,
 (2, 6): 0.0032,
 (2, 7): 0.0099,
 (2, 8): 0.0842,
 (2, 9): 0.0104,
 (3, 4): 0.0783,
 (3, 5): 0.6164,
 (3, 6): 0.0454,
 (3, 7): 0.1489,
 (3, 8): 1.38,
 (3, 9): 0.0646,
 (4, 5): 0.0582,
 (4, 6): 0.0025,
 (4, 7): 0.0125,
 (4, 8): 0.0787,
 (4, 9): 0.0121,
 (5, 6): 0.0401,
 (5, 7): 0.1307,
 (5, 8): 1.3607,
 (5, 9): 0.0557,
 (6, 7): 0.0021,
 (6, 8): 0.0181,
 (6, 9): 0.0015,
 (7, 8): 0.2602,
 (7, 9): 0.0131,
 (8, 9): 0.0707}